# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv('../output_data/citydata.csv')
city_data_df.head()
len(city_data_df)                           

561

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps.
gmaps.configure(api_key=g_key)

# Extract latitude and longitude
lat = city_data_df["Lat"].fillna(0)
lng = city_data_df["Lng"].fillna(0)

# Concatinate latitude and longitude and add it to a list
coordinates = list(zip(lat,lng))

figure_layout = {
    'width': '800px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin':'0 auto 0 auto'
    
}

fig = gmaps.figure(layout=figure_layout)

In [4]:
# Plot Heatmap layer with weight as Humidity  - Cities with higher humidity will be rendered with greater intensity
markers = gmaps.heatmap_layer(coordinates,weights = city_data_df['Humidity'])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down records that match criteria:
    # Cloudiness = 0, Max Temp  > 70 but < 90 and Wind Speed <= 10
condition = city_data_df[(city_data_df['Cloudiness'] > 0) | ((city_data_df['Max Temp'] <= 70) | (city_data_df['Max Temp'] >= 90)) | (city_data_df['Wind Speed'] > 10)]
narrowed_city_df = city_data_df.drop(condition.index, axis = 0, inplace= False)
len(narrowed_city_df)


6

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrowed_city_df.copy()

# Adding empty column to the dataframe
hotel_df["Hotel Name"] = ""

hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
54,Guararapes,0,BR,1584337050,89,-21.26,-50.64,72.10,3.89,
281,Fare,0,PF,1584337078,73,-16.70,-151.02,83.08,9.62,
417,Traralgon,0,AU,1584337097,56,-38.18,146.53,75.00,5.99,
437,Karibib,0,NaN,1584337099,42,-21.93,15.83,73.69,4.32,
444,Bahía Honda,0,CU,1584337100,84,22.90,-83.16,70.92,9.28,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
                   
params = {
    "location": "",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

#Using Google Places API to find the first hotel for each city located within 5000 meters of coordinates

for index, row in hotel_df.iterrows():
    params["location"] = str(row['Lat']) + "," + str(row['Lng'])
    response = requests.get(base_url, params=params).json()
    results = response['results']
    hotel_df.loc[index,'Hotel Name'] = results[0]['name']

In [8]:
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
54,Guararapes,0,BR,1584337050,89,-21.26,-50.64,72.10,3.89,Guararapes
281,Fare,0,PF,1584337078,73,-16.70,-151.02,83.08,9.62,Fare
417,Traralgon,0,AU,1584337097,56,-38.18,146.53,75.00,5.99,Traralgon
437,Karibib,0,NaN,1584337099,42,-21.93,15.83,73.69,4.32,Karibib
444,Bahía Honda,0,CU,1584337100,84,22.90,-83.16,70.92,9.28,Harlem


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:

heatmap_layer = gmaps.heatmap_layer(list(zip(locations['Lat'],locations['Lng'])))
popup_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure()

# Adding heatmap layer to the base layer
fig.add_layer(heatmap_layer)

# Add marker layer on top of heat map
fig.add_layer(popup_layer)
fig


Figure(layout=FigureLayout(height='420px'))